[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/milestones/07-DCT/block_DCT_compression.ipynb)

# Block-DCT (Discrete Cosine Transform) Image Compression

Compressing color images with PNG in the DCT domain. Remember to run [JPEG.ipynb](https://github.com/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/milestones/07-DCT/JPEG.ipynb) before!

## Global parameters of the notebook

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
!ln -sf ~/MRVC/src/image_3.py .
import image_3 as image
!ln -sf ~/MRVC/src/image_1.py .
import image_1 as component
!ln -sf ~/MRVC/src/block_DCT.py .
!ln -sf ~/MRVC/src/YCoCg.py .
import YCoCg as color
#!ln -sf ~/MRVC/src/color_DCT.py .
#import color_DCT as color
#!ln -sf ~/MRVC/src/RGB.py .
#import RGB as color
import cv2
!ln -sf ~/quantization/information.py .
import information
!ln -sf ~/quantization/distortion.py .
import distortion
import os
import pylab
!ln -sf ~/quantization/deadzone_quantizer.py .
import deadzone_quantizer as Q
import math
import block_DCT as DCT

In [ ]:
HOME = os.environ["HOME"]
#test_image = "../sequences/stockholm/"
test_image = HOME + "/MRVC/sequences/lena_color/"
#test_image = "../sequences/lena_bw/"

In [ ]:
block_y_side = block_x_side = 8

In [ ]:
N_components = 3

In [ ]:
entropy_estimator = "PNG"
if entropy_estimator == "PNG":
    def compute_BPP(_image, filename_prefix):
        BPP = image.write(_image, filename_prefix, 0)*8/_image.size
        return BPP
else:
    def compute_BPP(_image, filename_prefix=''):
        entropy = information.entropy(_image.flatten().astype(np.int16))
        return entropy

## Testing `block_DCT.analyze_block()` and `block_DCT.synthesize_block()`

The DCT concentrates the energy of the signal in a few coefficients.

In [ ]:
#a = np.random.randint(low=0, high=100, size=(4,4,3))
a = np.full(shape=(4,4,3), fill_value=10, dtype=np.int16)

In [ ]:
a

In [ ]:
b = DCT.analyze_block(a)

In [ ]:
b

In [ ]:
b.astype(np.int16)

In [ ]:
c = DCT.synthesize_block(b)

In [ ]:
c.astype(np.int16)

## Testing `DCT.analyze_image()` and `DCT.synthesize_image()`
Remember that an image is divided into blocks.

In [ ]:
x = image.read(test_image, 0)
image.show(x, title="Original")

In [ ]:
y = DCT.analyze_image(x, block_y_side, block_x_side)

In [ ]:
image.show(image.normalize(y), "Block DCT domain")
image.show(image.normalize(y[:64, :64]), "Block DCT domain (detail [0:64, 0:64])")

Again, most of the energy of each block has been concentrated in the low-pass frequency component (DC component).

In [ ]:
z = DCT.synthesize_image(y, block_y_side, block_x_side)

In [ ]:
r = x - z

In [ ]:
image.show(image.normalize(r), "DCT error when use integer truncation")

In [ ]:
image.show(z.astype(np.uint8), "Reconstructed image")

This error es generated by the truncation of the floating point coefficients (remember that we work with 16 bits integer) after the analysis, and also by the truncation of the floating point pixels after the synthesis. 

## Switching between blocks and subbands

The coefficients of the block-DCT can be reorganized in subbands. A subband with corrdinates (X, Y) is the 2D arragement of the coefficients that is in the coordinates (X, Y) of each block. The representation in subbands increases the spatial correlation between the coefficients (which also provides an improved visual comprehension of the content of the coefficients).

In [ ]:
x = image.read(test_image)
y = DCT.analyze_image(x, block_y_side, block_x_side)
y_subbands = DCT.get_subbands(y, block_y_side, block_x_side)

In [ ]:
image.show(image.normalize(y_subbands), f"Subbands of the {block_y_side}x{block_x_side} DCT domain")

In [ ]:
print(f"We have {block_y_side}x{block_x_side} subbands of {int(x.shape[0]/block_y_side)}x{int(x.shape[1]/block_x_side)} coefficients")

Reording the coefficients into subbands is completely reversible.

In [ ]:
yy = DCT.get_blocks(y_subbands, block_y_side, block_x_side)
(yy == y).all()

And, as it can be seen, the 2D correlation is higher in the low frequencies (left up corner) than in the high frequencies (right down corner).

In [ ]:
blocks_in_y = x.shape[0]//block_y_side
blocks_in_x = x.shape[1]//block_x_side
image.show(image.normalize(y_subbands[:blocks_in_y, :blocks_in_x]), f"Subband (0, 0) ({block_y_side}x{block_x_side} DCT)")

Subband (0,0) contains the low frequencies of the image.

In [ ]:
image.show(image.normalize(y_subbands[:blocks_in_y, blocks_in_x:2*blocks_in_x]), f"Subband (0, 1) ({block_y_side}x{block_x_side} DCT)")

The (0, 1) subband represents the slowest changes of the image in the horizontal domain.

In [ ]:
image.show(image.normalize(y_subbands[blocks_in_y:2*blocks_in_y, :blocks_in_x]), f"Subband (1, 0) ({block_y_side}x{block_x_side} DCT)")

The (1, 0) subband represents the slowest changes of the image in the vertical domain.

In [ ]:
image.show(image.normalize(y_subbands[blocks_in_y:2*blocks_in_y, blocks_in_x:2*blocks_in_x]), f"Subband (1, 1) ({block_y_side}x{block_x_side} DCT)")

The (1, 1) subband represents slowest changes in the diagonal (left up corner to right down corner) of the image.

## Subbands/components information

In [ ]:
x = image.read(test_image, 0)
#xx = color.from_RGB(x.astype(np.int16) - 128) # -128 decreases maximum value of the DC coefficients
######################################################
# This reduces the energy (not the entropy) of the    #
# subband-components compared to the previous option. #
# However, the averages should be encoded.            #
xx = color.from_RGB(x.astype(np.int16))              #
xx[...,0] -= np.average(xx[...,0]).astype(np.int16)  #
xx[...,1] -= np.average(xx[...,1]).astype(np.int16)  #
xx[...,2] -= np.average(xx[...,2]).astype(np.int16)  #
######################################################
yy = DCT.analyze_image(xx, block_y_side, block_x_side)
yy = DCT.get_subbands(yy, block_y_side, block_x_side)
print("   sb component maximum mininum max-min average std-dev entropy        energy  avg-enegy")
accumulated_entropy = 0
blocks_in_y = x.shape[0] // block_y_side
blocks_in_x = x.shape[1] // block_x_side
list_of_subbands_components = []
for _y in range(block_y_side):
    for _x in range(block_x_side):
        for _c in range(N_components):
            sbc = yy[blocks_in_y*_y:blocks_in_y*(_y+1),
                     blocks_in_x*_x:blocks_in_x*(_x+1),
                     _c]
            entropy = information.entropy(sbc.flatten().astype(np.int16))
            accumulated_entropy += entropy
            max = sbc.max()
            min = sbc.min()
            max_min = max - min
            avg = np.average(sbc)
            dev = math.sqrt(np.var(sbc))
            energy = information.energy(sbc)
            avg_energy = energy/sbc.size
            list_of_subbands_components.append((_y, _x, _c, max, min, max_min, avg, dev, entropy, energy, avg_energy))
            #print(f"{_y:2d} {_x:2d} {_c:9d} {max:7.1f} {min:7.1f} {max_min:7.1f} {avg:7.1f} {dev:7.1f} {entropy:7.1f} {energy:13.1f} {avg_energy:10.1f}")
sorted_list_of_subbands_components = sorted(list_of_subbands_components, key=lambda x: x[8])[::-1]
for _i in sorted_list_of_subbands_components:
    _y, _x, _c, max, _min, max_min, avg, dev, entropy, energy, avg_energy = _i
    print(f"{_y:2d} {_x:2d} {_c:9d} {max:7.1f} {min:7.1f} {max_min:7.1f} {avg:7.1f} {dev:7.1f} {entropy:7.1f} {energy:13.1f} {avg_energy:10.1f}")
avg_entropy = accumulated_entropy / (block_x_side * block_y_side * xx.shape[2])
print("Average entropy in the cosine domain:", avg_entropy)
print("Entropy in the image domain:", information.entropy(x.flatten().astype(np.uint8)))

As it can be observed, the 8x8-DCT accumulates most of the energy (and entropy) in the low-frequency subbands.

## Quantization steps

Considering the previous dynamic range values for the YCoCg/8x8-DCT coefficients, this parameter should allow to use 8 bits/pixel images, if we are using PNG as an entropy codec. As it can be seen, we need 11 bits for representing the DC coefficients (notice that we have substracted 128 to the image to reduce in one bit the number of bits of the low frequency subband) and after quantization, we should use only 8. Therefore, the minimum quantization step should be 1<<3 = 8. Notice that 11 - 8 = 3.

In [ ]:
Q_steps = [128, 64, 32, 16, 8]

## Testing `DCT.uniform_quantize()` and `DCT.uniform_dequantize()` NO
Quantization removes information but also increases the compression ratios of the stored images. These methods quantize all coefficients with the same quantization step.

In [ ]:
Q_step = 128
x = image.read(test_image, 0)
xx = color.from_RGB(x.astype(np.int16) - 128)
yy = DCT.analyze_image(xx, block_y_side, block_x_side)
yy = DCT.get_subbands(yy, block_y_side, block_x_side)
yy_k = DCT.uniform_quantize(yy, block_y_side, block_x_side, N_components, Q_step)
yy_dQ = DCT.uniform_dequantize(yy_k, block_y_side, block_x_side, N_components, Q_step)
yy_dQ = DCT.get_blocks(yy_dQ, block_y_side, block_x_side)
zz_dQ = DCT.synthesize_image(yy_dQ, block_y_side, block_x_side)
z_dQ = color.to_RGB(zz_dQ) + 128
image.show(np.clip(z_dQ, a_min=0, a_max=255), f"Quantized image (Q_step={Q_step}) in the {block_y_side}x{block_x_side} DCT {color.name} domain")

In [ ]:
r = x - z_dQ
n = image.normalize(r)
image.show(n, "Quantization error")

Thereore, quantization in the DCT domain tends to remove high frequencies (it works basically as a low pass filter).

## Coding subbands vs coding blocks
Let's see the effect of encoding the DCT coefficients grouped by subbands. For simplicity, we will use uniform quantization. We compare compressing the transform image: (1) by blocks (the output of each block-DCT) and (2) by subbands.

In [ ]:
x = image.read(test_image, 0)
xx = color.from_RGB(x.astype(np.int16) - 128)

RD_points_blocks = []
RD_points_subbands = []
for Q_step in Q_steps:
    yy = DCT.analyze_image(xx, block_y_side, block_x_side)
    # Notice that with uniform_quantize() does not matter if the DCT domain
    # is organized in subbands or blocks.
    yy_k = DCT.uniform_quantize(yy, block_y_side, block_x_side, N_components, Q_step)
    #BPP, _y_Q = information.PNG_BPP((y_Q.astype(np.int32) + 32768).astype(np.uint16), f"/tmp/{Q_step}_")
    #assert (y_Q == _y_Q.astype(np.int32) - 32768).all()
    BPP = image.write((yy_k + 128).astype(np.uint8), f"/tmp/{Q_step}_", 0)*8/xx.size
    
    # Check that we can recover the code-stream #########
    __ = image.read(f"/tmp/{Q_step}_", 0)               #
    try:                                                #
        assert ((yy_k + 128) == __).all()               #
    except AssertionError:                              #
        counter = 0                                     #
        for _i in range(x.shape[0]):                    #
            for _j in range(x.shape[1]):                #
                if (yy_k[_i, _j] != __[_i, _j]).any():  #
                    print(yy_k[_i, _j], __[_i, _j])     #
                    if counter > 10:                    #
                        break                           #
                    counter += 1                        #
            if counter > 10:                            #
                break                                   #
    #####################################################
    yy_dQ = DCT.uniform_dequantize(yy_k, block_y_side, block_x_side, N_components, Q_step)
    zz_dQ = DCT.synthesize_image(yy_dQ, block_y_side, block_x_side)
    z_dQ = color.to_RGB(zz_dQ) + 128
    # Notice that to compute the distortion, the DCT domain could be
    # also used because the DCT is unitary.
    MSE = distortion.MSE(x, z_dQ)
    RD_points_blocks.append((BPP, MSE))
    yy_k_subbands = DCT.get_subbands(yy_k, block_y_side, block_x_side)
    #BPP, _yy_k_subbands = information.PNG_BPP((yy_k_subbands.astype(np.int32) + 32768).astype(np.uint16), f"/tmp/s_{Q_step}_")
    #assert (yy_k_subbands == _yy_k_subbands.astype(np.int32) - 32768).all()
    #BPP = image.write((yy_k_subbands + 128).astype(np.uint8), f"/tmp/{Q_step}_", 0)*8/xx.size
    BPP = compute_BPP((yy_k_subbands + 128).astype(np.uint8), f"/tmp/{Q_step}_")
    # Check that we can recover the code-stream #################
    __ = image.read(f"/tmp/{Q_step}_", 0)                       #
    try:                                                        #
        assert ((yy_k_subbands + 128) == __).all()              #
    except AssertionError:                                      #
        counter = 0                                             #
        for _i in range(x.shape[0]):                            #
            for _j in range(x.shape[1]):                        #
                if (yy_k_subbands[_i, _j] != __[_i, _j]).any(): #
                    print(yy_k_subbands[_i, _j], __[_i, _j])    #
                    if counter > 10:                            #
                        break                                   #
                    counter += 1                                #
            if counter > 10:                                    # 
                break                                           #
    #############################################################
    RD_points_subbands.append((BPP, MSE))
    print(Q_step, end=' ', flush=True)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points_blocks), label=f"{block_y_side}x{block_x_side} DCT (encoded by blocks)")
pylab.plot(*zip(*RD_points_subbands), label=f"{block_y_side}x{block_x_side} DCT (encoded by subbands)")
pylab.title("")
pylab.xlabel("BPP")
pylab.ylabel("MSE")
plt.legend(loc="best")
pylab.show()

Coding by subbands is more efficient because PNG can exploit better the spatial correlation between the coefficients.

## Can we do it better?

Let's compute the optimal sequence of quantization steps for the set of possible combinations of subbands and components. We will compute the distortion of each subband-component in the YCoCg/8x8-DCT domain for a set of quantization steps, considering that the YCoCg transform is near-orthogonal and that the 8x8-DCT is full-orthogonal. Thanks to orthogonality, we can assume that the quantization error generated in one subband does not influence on the quantization error added to the other subbands because the DCT coefficients are uncorrelated, or in other words, that the quantization error generated in one coefficient (or subband) is not correlated with the quantization error generated in other coefficients (or subbands).

Algorithm:
1. Read the image.
2. Transform it to the YCoCg domain.
3. Transform each YCoCg component to the 8x8-DCT domain.
4. Find a set RD points for each subband-component.
5. Compute the slope of each point and put all the slopes in the same list.
6. Sort the previous list by the slope field.
7. Find the RD curve that progressively uses smaller slopes.

## Read the image and move to the YCoCg domain

In [ ]:
x = image.read(test_image, 0)
#xx = color.from_RGB(x.astype(np.int16) - 128)
xx = color.from_RGB(x.astype(np.int16))
xx[...,0] -= np.average(xx[...,0]).astype(np.int16)
xx[...,1] -= np.average(xx[...,1]).astype(np.int16)
xx[...,2] -= np.average(xx[...,2]).astype(np.int16)

## Move each component to the 8x8-DCT domain

In [ ]:
yy = DCT.analyze_image(xx, block_y_side, block_x_side)
yy = DCT.get_subbands(yy, block_y_side, block_x_side)

## Find the slope of each quantization step for each subband-component
Create a list per subband-component of RD points and a list per subband-component of RD slopes. The first RD point is computed for 0 BPP, where the MSE distortion is equal to the average energy of the subband-component (notice that the average of each subband-component should be 0).

In [ ]:
RD_points = []
RD_slopes = []
N_components = xx.shape[2]
for _y in range(block_y_side):
    for _x in range(block_x_side):
        for _c in range(N_components):
            sbc = yy[blocks_in_y*_y : blocks_in_y*(_y + 1),
                     blocks_in_x*_x : blocks_in_x*(_x + 1),
                     _c]
            sbc_energy = information.average_energy(sbc)
            # The first point of each RD curve has a maximum distortion equal
            # to the energy of the subband and a rate = 0
            RD_points.append([(0, sbc_energy)])
            RD_slopes.append([])

In [ ]:
RD_points # (BPP, MSE)

Now populate the rest of points of each subband-component. **Distortion is estimated in the transform domain** supposing that both, the color and the spatial (DCT) transforms are orthogonal. Notice also that we consider only the length of the subband-component.

In [ ]:
for _y in range(block_y_side):
    for _x in range(block_x_side):
        for _c in range(N_components):
            sbc = yy[blocks_in_y*_y : blocks_in_y*(_y + 1),
                     blocks_in_x*_x : blocks_in_x*(_x + 1),
                    _c]
            counter = 0
            for Q_step in Q_steps:
                sbc_k = Q.quantize(sbc, Q_step)
                sbc_dQ = Q.dequantize(sbc_k, Q_step)
                MSE = distortion.MSE(sbc, sbc_dQ)
                BPP = component.write(sbc_k.astype(np.uint8), f"/tmp/{_y}_{_x}_{Q_step}_", 0)*8/xx.size
                #BPP_Q_indexes = information.PNG_BPP((Q_indexes.astype(np.int32) + 32768).astype(np.uint16), "/tmp/BPP_")[0]
                #BPP_Q_indexes = information.entropy(Q_indexes.astype(np.int16).flatten())
                point = (BPP, MSE)
                RD_points[(_y * block_x_side * N_components + _x * N_components ) + _c].append(point)
                print("Q_step =", Q_step, "BPP =", point[0], "MSE =", point[1])
                delta_BPP = BPP - RD_points[(_y * block_x_side * N_components + _x * N_components ) + _c][counter][0]
                delta_MSE = RD_points[(_y * block_x_side * N_components + _x * N_components ) + _c][counter][1] - MSE
                if delta_BPP > 0:
                    slope = delta_MSE/delta_BPP
                    RD_slopes[(_y * block_x_side * N_components + _x * N_components) + _c].append((slope, (_y, _x, _c), Q_step))
                else:
                    slope = 0
                #RD_slopes[(_y * block_x_side * N_components + _x * N_components) + _c].append((Q_step, slope, (_y, _x, _c)))
                counter += 1

In [ ]:
RD_points

In [ ]:
RD_slopes # (Qstep, slope, subband-component_index)

## Remove points that do not belong to the convex-hull

In [ ]:
def filter_slopes(slopes):
    filtered_slopes = []
    slopes_iterator = iter(slopes)
    prev = next(slopes_iterator)
    for curr in slopes_iterator:
        if prev[0] < curr[0]:
            print(f"deleted {prev}")
        else:
            filtered_slopes.append(prev)
        prev = curr
    filtered_slopes.append(prev)
    return filtered_slopes

filtered_slopes = []
for i in RD_slopes:
    filtered_slopes.append(filter_slopes(i))

In [ ]:
filtered_slopes

## Sort the RD points by their slope

In [ ]:
single_list = []
for l in filtered_slopes:
    #l = filter_slopes(l)
    for i in l:
        #if i[1] > 0:
        single_list.append(i)

In [ ]:
single_list

In [ ]:
sorted_slopes = sorted(single_list, key=lambda x: x[0])[::-1]

In [ ]:
sorted_slopes

## Build the optimal RD curve
We use the sorted list of slopes (with quantization and subband-component information) to generate the optimal RD list of RD points. Notice that, although the YCoCg components gains ($\frac{3}{2}\Delta_{\text{Y}} = \Delta_{\text{Co}} = \frac{3}{2}\Delta_{\text{Cg}}$) have not been taken into consideration, when the RD points are sorted by their slope, this information has been taken into consideration.

In [ ]:
optimal_RD_points = []
yy_prog = np.zeros_like(yy)
Q_steps_combination = np.full(shape=(block_x_side, block_y_side, N_components), fill_value=99999999)
for s in sorted_slopes:
    sbc_index = s[1]
    _y = sbc_index[0]
    _x = sbc_index[1]
    _c = sbc_index[2]
    Q_steps_combination[_y, _x, _c] = s[2]
    #print(sbc_index, Q_steps_combination[_y, _x, _c])
    yy_prog[blocks_in_y*_y : blocks_in_y*(_y + 1),
            blocks_in_x*_x : blocks_in_x*(_x + 1),
            _c] = yy[blocks_in_y*_y : blocks_in_y*(_y + 1),
                     blocks_in_x*_x : blocks_in_x*(_x + 1),
                     _c]
    yy_prog_k = DCT.quantize(yy_prog, Q_steps_combination)
    yy_prog_dQ = DCT.dequantize(yy_prog_k, Q_steps_combination)
    
    # Uncomment the following line to measure the distortion in the color_transform+DCT domain
    MSE = distortion.MSE(yy, yy_prog_dQ)

    # Uncomment the following 3 lines to measure the distortion in the color_transform domain
    #yy_prog_dQ = DCT.get_blocks(yy_prog_dQ, block_y_side, block_x_side)
    #zz_prog = DCT.synthesize_image(yy_prog_dQ, block_y_side, block_x_side)
    #MSE = distortion.MSE(xx, zz_prog)
    
    # If the color transform domain is not orthogonal, the MSE should be measured in the RGB domain

    #BPP, _y_quant = information.PNG_BPP((y_quant.astype(np.int32) + 32768).astype(np.uint16), f"/tmp/{i}_{j}_{Q_step}_")
    #assert (y_quant == _y_quant.astype(np.int32) - 32768).all()
    BPP = image.write((yy_prog_k + 128).astype(np.uint8), f"/tmp/{_y}_{_x}_{_c}_{s[0]}_", 0)*8/xx.size
    point = (BPP, MSE)
    print("sbc =", sbc_index, "Q_step =", s[2], "BPP =", BPP, "MSE =", MSE)
    optimal_RD_points.append(point)

## Read JPEG RD data to compare
Notice that the chroma has not been subsampled.

In [ ]:
JPEG_RD_points = []
with open("JPEG.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        JPEG_RD_points.append((float(rate), float(_distortion)))

In [ ]:
#DCT2 = []
#with open("DCT.txt", 'r') as f:
#    for line in f:
#        rate, _distortion = line.split('\t')
#        DCT2.append((float(rate), float(_distortion)))

## Compare

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points_subbands), label="uniform quantization")
pylab.plot(*zip(*optimal_RD_points), label="optimal quantization")
pylab.plot(*zip(*JPEG_RD_points), label="JPEG")
#pylab.plot(*zip(*DCT2), label="old")
pylab.title("Comparing with JPEG")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc="best")
#pylab.yscale('log')
#pylab.xscale('log')
pylab.show()

In [ ]:
with open('DCT.txt', 'w') as f:
    for item in optimal_RD_points:
        f.write(f"{item[0]}\t{item[1]}\n")

## Conclusions
The use of RD optimization increases the performance of the (color_transform + spatial_transform + entropy coding)-procedure, and more concretely of YCoCg + NxN-DCT + PNG. This is a consequence of selecting those quantization steps for the subband-components that contribute more to the quality of the reconstruction.